# Multi-Asset Simulation - Floods
The following notebook contains an example of running the PRISK algorithm on a collection of assets. The inclusion of multiple assets introduces interactions because of the insurance link between assets who share the same insurance.

In [1]:
import numpy as np
import pandas as pd

from prisk.flood import FloodExposure, FloodEntitySim
from prisk.kernel import Kernel
from prisk.asset import PowerPlantSim
from prisk.insurance import Insurance
from prisk.utils import convert_to_continous_damage

In [2]:
TIME_HORIZON = 10
n_assets = 100
n_insurers = 1

## Data description
The same type of data is required as for the previous notebooks.

In [3]:
power = pd.read_excel("../Data/power.xlsx")
power.drop_duplicates(subset=["Plant / Project name", "Type", "Capacity (MW)", "Country", "Owner"], inplace=True)
power.drop(columns=[2], inplace=True)
power = power#.sample(n_assets, axis=0)
n_assets = power.shape[0]
damage_curves = pd.read_excel("../Data/damage_curves.xlsx")
continuous_curves = convert_to_continous_damage(damage_curves)
return_period_columns = [c for c in power.columns if isinstance(c, int)]

## Simulation
The simulation will consist of 20 assets and 2 insurers. Each firm is assumed to have the same damage curves.

In [4]:
assets = []
insurers = []

for i in range(n_insurers):
    insurer = Insurance(
        name=f"Insurer {i}",
        capital=1e9,
    )
    insurers.append(insurer)

for a in range(n_assets):
    firm = power.iloc[a]
    flood_exposure = [FloodExposure(return_period, firm[return_period]) 
                    for return_period in return_period_columns if firm[return_period] > 0
    ]
    production_path = np.repeat(firm["Capacity (MW)"]*24*365, TIME_HORIZON) # Assume constant production path
    asset = PowerPlantSim(
        name=firm["Plant / Project name"],
        flood_damage_curve=continuous_curves,
        flood_exposure=flood_exposure,
        production_path=production_path,
        replacement_cost=firm["Value"],
    )
    # Give them a random insurer
    asset.add_insurer(np.random.choice(insurers))
    assets.append(asset)

In [5]:
kernel = Kernel(assets=assets, insurers=insurers)
for asset in assets:
    FloodEntitySim(asset).simulate(time_horizon=TIME_HORIZON, kernel=kernel)
kernel.run(time_horizon=TIME_HORIZON)

## Monte Carlo Sim
Here the monte carlo simulation is done on all of the assets at the same time.

In [6]:
simulations = 250
prisks = pd.DataFrame(columns=[f"Asset {i}" for i in range(n_assets)], 
                      index=[i for i in range(simulations)])
components = pd.DataFrame(
    columns=["asset", "run", "base_value", "replacement",
             "business_disruption", "fair_insurance",
             "insurance_adjustment", "npv", "prisk"]
)


from tqdm import tqdm
for i in tqdm(range(simulations)):
    assets = []
    insurers = []

    for k in range(n_insurers):
        insurer = Insurance(
            name=f"Insurer {k}",
            capital=1e9,
        )
        insurers.append(insurer)

    for a in range(n_assets):
        firm = power.iloc[a]
        flood_exposure = [FloodExposure(return_period, firm[return_period]) 
                        for return_period in return_period_columns if firm[return_period] > 0
        ]
        production_path = np.repeat(firm["Capacity (MW)"]*24*365, TIME_HORIZON) # Assume constant production path
        asset = PowerPlantSim(
            name=firm["Plant / Project name"],
            flood_damage_curve=continuous_curves,
            flood_exposure=flood_exposure,
            production_path=production_path.copy(),
            replacement_cost=firm["Value"],
        )
        # Give them a random insurer
        #asset.add_insurer(np.random.choice(insurers))
        assets.append(asset)
    kernel = Kernel(assets=assets, insurers=insurers)
    for asset in assets:
        FloodEntitySim(asset).simulate(time_horizon=TIME_HORIZON, kernel=kernel)
    kernel.run(time_horizon=TIME_HORIZON, verbose=0)
    for j, asset in enumerate(assets):
        prisk = max(asset.npv/asset.base_value, 0)
        prisks.loc[i, f'Asset {j}'] = prisk
    component = pd.DataFrame({
            "asset": asset.name,
            "run": i,
            "base_value": asset.base_value,
            "replacement": asset.total_replacement_costs,
            "business_disruption": asset.total_business_disruption,
            "fair_insurance": asset.total_fair_insurance_premiums,
            "insurance_adjustment": asset.total_insurance_adjustments,
            "npv": asset.npv,
            "prisk": max(asset.npv/asset.base_value, 0)
        } for asset in assets)
    components = pd.concat([components, component], ignore_index=True)
prisks

  0%|          | 0/250 [00:00<?, ?it/s]/var/folders/zz/zvsv6f4s4wn29k9lt76bdlsr0000gn/T/ipykernel_61984/770774891.py:57: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  components = pd.concat([components, component], ignore_index=True)
100%|██████████| 250/250 [18:26<00:00,  4.43s/it]


,Asset 0,Asset 1,Asset 2,Asset 3,Asset 4,Asset 5,Asset 6,Asset 7,Asset 8,Asset 9,...,Asset 11553,Asset 11554,Asset 11555,Asset 11556,Asset 11557,Asset 11558,Asset 11559,Asset 11560,Asset 11561,Asset 11562
0,1.0,0,1.0,1.0,1.0,1.0,1.0,1.0,0.620057,1.0,...,1.0,0.967723,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
1,1.0,0,1.0,1.0,1.0,1.0,1.0,1.0,0.599851,1.0,...,1.0,0.843993,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
2,1.0,0.639054,1.0,1.0,1.0,1.0,1.0,1.0,0.619858,1.0,...,1.0,0.974153,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
3,1.0,0.60006,1.0,1.0,1.0,1.0,1.0,1.0,0.645056,1.0,...,1.0,0.934056,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
4,1.0,0.620057,1.0,1.0,1.0,1.0,1.0,1.0,0.60006,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245,1.0,0.377809,1.0,1.0,1.0,1.0,1.0,1.0,0.63389,1.0,...,1.0,0.843993,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
246,0.703467,0.639054,1.0,1.0,1.0,1.0,1.0,1.0,0.734672,1.0,...,1.0,0.879285,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
247,1.0,0.220117,1.0,1.0,1.0,1.0,1.0,1.0,0.586713,1.0,...,1.0,0.923848,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0
248,0.732379,0.620057,1.0,1.0,1.0,1.0,1.0,1.0,0.60006,1.0,...,1.0,0.939389,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0


In [7]:
power.loc[:, 'prisk'] = list(prisks.mean(axis=0))
power.drop(columns=["Latitude", "Longitude"], inplace=True)
power.to_parquet("../results/240617/power_with_prisk_non_insured.parquet.gzip", compression="gzip")
components.to_parquet("../results/240617/components_non_insured.parquet.gzip", compression="gzip")
prisks.to_parquet("../results/240617/prisks_non_insured.parquet.gzip", compression="gzip")


/Users/rubenkerkhofs/Documents/Oxford/PRISK/.env/lib/python3.10/site-packages/pandas/io/parquet.py:190: UserWarning: The DataFrame has column names of mixed type. They will be converted to strings and not roundtrip correctly.
  table = self.api.Table.from_pandas(df, **from_pandas_kwargs)
